In [ ]:
# https://github.com/Newmu/Theano-Tutorials
# see batch normalization

In [1]:
import datasets

datasets.downloads = '/home/greg/Downloads/'

trX, teX, trY, teY = datasets.mnist()

In [2]:
import numpy as np
from softmax import softmax
from scipy.special import expit

In [12]:
X = trX
Y = trY

epochs = 1
learning_rate = 0.05
H = 625

N, D = X.shape
N, C = Y.shape

def accuracy(yhat):
    decode = lambda y: np.argmax(y, axis = 1)
    return np.mean(decode(yhat) == decode(Y[:len(yhat)]))

In [14]:
import theano
from theano import tensor as T

x = T.fmatrix()
y = T.fmatrix()

weights = lambda shape: theano.shared(np.random.normal(size = shape))

w_h = weights([D, H])
w_o = weights([H, C])

cost = T.mean(T.nnet.categorical_crossentropy(T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(x, w_h)), w_o)), y))
updates = [[p, p - T.grad(cost, p) * learning_rate] for p in [w_h, w_o]]
optimizer = theano.function([x, y], updates = updates, allow_input_downcast = True)

for _ in range(epochs):
    for start, end in zip(range(0, N, 128), range(100, N, 100)):
        optimizer(X[start : end], Y[start : end])
        
w_h, w_o = w_h.get_value(), w_o.get_value()
print accuracy(softmax(expit(X[:1000].dot(w_h)).dot(w_o)))

0.747


In [87]:
import tensorflow as tf

x = tf.placeholder('float')
y = tf.placeholder('float')

weights = lambda shape: tf.Variable(np.random.normal(size = shape).astype(np.float32))
params = []

w = weights([D, H])
b = weights([H])
l = tf.nn.relu(tf.matmul(x, w) + b)
params += [w, b]

w = weights([H, C])
b = weights([C])
model = tf.matmul(l, w) + b
params += [w, b]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model, y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    for _ in range(epochs):
        for start, end in zip(range(0, N, 100), range(100, N, 100)):
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

    params = [p.eval() for p in params]
    print accuracy(model.eval({x: X[:1000]}))

0.925


In [84]:
xx = relu(X[:5].dot(w1) + b1).dot(w2) + b2
yy = Y[:5]
x = tf.matmul(tf.nn.relu(tf.matmul(X[:5], w1.astype(np.float64)) + b1), w2.astype(np.float64)) + b2
y = tf.constant(yy)
with tf.Session() as sess:
    print sess.run(tf.nn.softmax_cross_entropy_with_logits(x, y))
    
from softmax import softmax_cross_entropy_with_logits
print softmax_cross_entropy_with_logits(xx, yy)

[  1.73340135e-06   0.00000000e+00   0.00000000e+00   0.00000000e+00
   6.96579426e-03]
[  1.73340135e-06  -0.00000000e+00  -0.00000000e+00  -0.00000000e+00
   6.96579426e-03]


In [85]:
relu = lambda x: x * (x > 0)

w1, b1, w2, b2 = params

from softmax import softmax_cross_entropy_with_logits

#print accuracy(softmax(expit(relu(X[:1000].dot(w1) + b1).dot(w2) + b2)))
accuracy(softmax_cross_entropy_with_logits(relu(X[:1000].dot(w1) + b1).dot(w2) + b2, Y[:1000]))

ValueError: axis(=1) out of bounds

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(output_dim = H, input_dim = D))
model.add(Activation('relu'))
model.add(Dense(output_dim = C))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])
model.fit(X, Y, nb_epoch = 1, batch_size = 100)
c = model.predict_classes(X[:1000], batch_size = 100)
np.mean(c == np.argmax(Y[:len(c)], axis = 1))

Epoch 1/1
 900/1000 [==========================>...] - ETA: 0s

0.872